# Figure 1 (Initial Data Analysis)

### Standard imports and loading data

In [3]:
import pandas as pd
import numpy as np
from PayneLabData import BurkholderiaTimeCourse as btc
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.layouts import row, column
from bokeh.palettes import Viridis8
from bokeh.models import Span
output_notebook()

Loading BokehJS ...

In [2]:
!pip install bokeh

    100% |████████████████████████████████| 16.3MB 1.3MB/s eta 0:00:01
  Stored in directory: /root/.cache/pip/wheels/7c/cc/1c/a8f447869feb88844eed88b9524f1974ff6219c9b586b3bfab
Successfully built bokeh
You are using pip version 19.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


## B_thai No Glucose Analysis

In [31]:
bthai = btc.raw_data

In [47]:
index_as_list = bthai.index.tolist()
new_index_list = list()
for prot in index_as_list:
    new_index_list.append(prot.split('|')[1])
bthai.index = new_index_list

In [79]:
for index in bthai.index:
    if index[0] == 'A':
        print(index)

A0A096YH13
A0A096YH16
A0A096YH24
A0A096YH26
A0A096YH27
A0A096YH29
A0A096YH30
A0A096YH33
A0A096YH36
A0A096YH40
A0A096YH43
A0A096YH44
A0A096YH58
A0A096YH64
A0A096YH77
A0A096YH87
A0A096YH97
A0A096YHA2
A0A096YHA3
A0A096YHB2
A0A096YHB3
A0A096YHB9
A0A096YHC1
A0A096YHC2
A0A096YHC9
A0A096YHD1
A0A096YHD8
A0A096YHE0
A0A096YHE5
A0A096YHE9
A0A096YHF0
A0A096YHG8
A0A096YHH5
A0A096YHH8
A0A096YHI1
A0A096YHI8
A0A096YHJ5
A0A096YHK6
A0A096YHK7
A0A096YHL0
A0A096YHM0
A0A096YHM1
A0A096YHM7
A0A096YHN0
A0A096YHP3
A0A096YHP5
A0A096YHP8
A0A096YHR8
A0A096YHT5
A0A096YHT6
A0A096YHU0
A0A096YHU2
A0A096YHU7
A0A096YHU9
A0A096YHV0
A0A096YHV2
A0A096YHV8
A0A096YHW0
A0A096YHW1
A0A096YHW2
A0A096YHW6
A0A096YHX2
A0A096YHX4
A0A096YHX9
A0A096YHY1
A0A096YHY4
A0A096YHY5
A0A096YHZ6
A0A096YHZ7
A0A096YI00
A0A096YI06
A0A096YI20
A0A096YI23
A0A096YI27
A0A096YI31
A0A096YI32
A0A096YI33
A0A096YI35
A0A096YI41
A0A096YI49
A0A096YI51
A0A096YI52
A0A096YI56
A0A096YI73
A0A096YI75
A0A096YI78
A0A096YI79
A0A096YI91
A0A096YI93
A0A096YI95
A0A096YI97

### Find standard biological variability by using the standard deviations of each time point

In [48]:
bthai_stdev = pd.DataFrame()
bthai_stdev['tp1'] = np.log(bthai[['Glc(-)_T11', 'Glc(-)_T12', 'Glc(-)_T13', 'Glc(-)_T14']].std(axis=1)).replace([np.inf, -np.inf], 0)
bthai_stdev['tp2'] = np.log(bthai[['Glc(-)_T21', 'Glc(-)_T22', 'Glc(-)_T23', 'Glc(-)_T24']].std(axis=1)).replace([np.inf, -np.inf], 0)
bthai_stdev['tp3'] = np.log(bthai[['Glc(-)_T31', 'Glc(-)_T32', 'Glc(-)_T33', 'Glc(-)_T34']].std(axis=1)).replace([np.inf, -np.inf], 0)
bthai_stdev['tp4'] = np.log(bthai[['Glc(-)_T41', 'Glc(-)_T42', 'Glc(-)_T43', 'Glc(-)_T44']].std(axis=1)).replace([np.inf, -np.inf], 0)
bthai_stdev['tp5'] = np.log(bthai[['Glc(-)_T51', 'Glc(-)_T52', 'Glc(-)_T53', 'Glc(-)_T54']].std(axis=1)).replace([np.inf, -np.inf], 0)
bthai_stdev = bthai_stdev.dropna(axis = 0)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in log
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in log
  


### Find the typical biological variability--the number below which 99% of standard deviations (across all time points) lie

In [49]:
all_stdev = pd.melt(bthai_stdev)['value'].tolist()
all_stdev.sort()
logcutoff = all_stdev[int(len(all_stdev)*0.99)]
cutoff = 2**logcutoff
print(cutoff)

1.4908900090837327


## Plot all standard deviations combined

In [50]:
# Build the basic figure
std_p = figure(width=1000, plot_height=1000, title = 'Distribution of Standard Deviations for All Time Points',
                  x_axis_label = 'Standard Deviation (Log Scale)',
                  y_axis_label = 'Number of Proteins')

# Create the histogram and add it to the graph
tp1_std_hist, edges = np.histogram(all_stdev, density=False, bins=100)

# Add each histogram to the graph
std_p.quad(top=tp1_std_hist, bottom=0, left=edges[:-1], right=edges[1:], fill_alpha=0.70, line_color='white')

# Add significance cutoff line
vline = Span(location = logcutoff, dimension = 'height', line_width=2)
std_p.renderers.extend([vline])

# Plot Styling
std_p.xgrid.visible = False
std_p.ygrid.visible = False
std_p.xaxis.minor_tick_line_color = None
std_p.yaxis.minor_tick_line_color = None
std_p.legend.click_policy = 'hide'
std_p.title.text_font_size = '25pt'
std_p.title.align = 'center'
std_p.xaxis.axis_label_text_font_size = '30pt'
std_p.yaxis.axis_label_text_font_size = '30pt'
std_p.xaxis.major_label_text_font_size = "15pt"
std_p.yaxis.major_label_text_font_size = "15pt"
std_p.legend.label_text_font_size = '15pt'

show(std_p)

/usr/local/lib/python3.6/dist-packages/bokeh/models/plots.py:725: UserWarning: 
You are attemptings to set `plot.legend.click_policy` on a plot that has zero legends added, this will have no effect.

Before legend properties can be set, you must add a Legend explicitly, or call a glyph method with the 'legend' parameter set.

  warnings.warn(_LEGEND_EMPTY_WARNING % attr)
/usr/local/lib/python3.6/dist-packages/bokeh/models/plots.py:725: UserWarning: 
You are attemptings to set `plot.legend.label_text_font_size` on a plot that has zero legends added, this will have no effect.

Before legend properties can be set, you must add a Legend explicitly, or call a glyph method with the 'legend' parameter set.

  warnings.warn(_LEGEND_EMPTY_WARNING % attr)


## Plot all the standard deviations on top of each other

In [51]:
# Build the basic figure
std_p = figure(width=1000, plot_height=1000, title = 'Distribution of Standard Deviations at Each Time Point',
                  x_axis_label = 'Standard Deviation (Log Scale)',
                  y_axis_label = 'Number of Proteins')

# Create the histograms for each time point
tp1_std_hist, edges1 = np.histogram(bthai_stdev['tp1'], density=False, bins=100)
tp2_std_hist, edges2 = np.histogram(bthai_stdev['tp2'], density=False, bins=100)
tp3_std_hist, edges3 = np.histogram(bthai_stdev['tp3'], density=False, bins=100)
tp4_std_hist, edges4 = np.histogram(bthai_stdev['tp4'], density=False, bins=100)
tp5_std_hist, edges5 = np.histogram(bthai_stdev['tp5'], density=False, bins=100)

# Add each histogram to the graph
for data, edge, name, color in zip([tp1_std_hist, tp2_std_hist, tp3_std_hist, tp4_std_hist, tp5_std_hist], [edges1, edges2, edges3, edges4, edges5], ['Time Point 1', 'Time Point 2', 'Time Point 3', 'Time Point 4', 'Time Point 5'], Viridis8):
    std_p.quad(top=data, bottom=0, left=edge[:-1], right=edge[1:], color=color, fill_alpha = 0.25, legend=name)

# Add significance cutoff line
vline = Span(location = logcutoff, dimension = 'height', line_width=2)
std_p.renderers.extend([vline])

# Plot Styling
std_p.xgrid.visible = False
std_p.ygrid.visible = False
std_p.xaxis.minor_tick_line_color = None
std_p.yaxis.minor_tick_line_color = None
std_p.legend.click_policy = 'hide'
std_p.title.text_font_size = '25pt'
std_p.title.align = 'center'
std_p.xaxis.axis_label_text_font_size = '30pt'
std_p.yaxis.axis_label_text_font_size = '30pt'
std_p.xaxis.major_label_text_font_size = "15pt"
std_p.yaxis.major_label_text_font_size = "15pt"
std_p.legend.label_text_font_size = '15pt'

show(std_p)

## Calculate the means for each time point

In [52]:
bthai_mean = pd.DataFrame()
bthai_mean['tp1'] = bthai[['Glc(-)_T11', 'Glc(-)_T12', 'Glc(-)_T13', 'Glc(-)_T14']].mean(axis=1)
bthai_mean['tp2'] = bthai[['Glc(-)_T21', 'Glc(-)_T22', 'Glc(-)_T23', 'Glc(-)_T24']].mean(axis=1)
bthai_mean['tp3'] = bthai[['Glc(-)_T31', 'Glc(-)_T32', 'Glc(-)_T33', 'Glc(-)_T34']].mean(axis=1)
bthai_mean['tp4'] = bthai[['Glc(-)_T41', 'Glc(-)_T42', 'Glc(-)_T43', 'Glc(-)_T44']].mean(axis=1)
bthai_mean['tp5'] = bthai[['Glc(-)_T51', 'Glc(-)_T52', 'Glc(-)_T53', 'Glc(-)_T54']].mean(axis=1)
bthai_mean = bthai_mean.dropna(axis = 0)

## Cluster the proteins (manually) based on increasing/decreasing/no change/spike at time point

In [53]:
cutoff_95 = 0.9007950046165919
cutoff_99 = 1.7792177941476046

bthai_mean['max'] = bthai_mean[['tp1', 'tp2', 'tp3', 'tp4', 'tp5']].max(axis=1)
bthai_mean['min'] = bthai_mean[['tp1', 'tp2', 'tp3', 'tp4', 'tp5']].min(axis=1)
bthai_mean['diff_max_min'] = bthai_mean['max'] - bthai_mean['min']
bin_rejects = bthai_mean.loc[bthai_mean['diff_max_min'] < cutoff]
significant_change = bthai_mean.loc[bthai_mean['diff_max_min'] >= cutoff]
significant_change['max'] = significant_change[['tp1', 'tp2', 'tp3', 'tp4', 'tp5']].max(axis=1)

bin_1 = significant_change[significant_change['tp1'] == significant_change['max']]
bin_2 = significant_change[significant_change['tp2'] == significant_change['max']]
bin_3 = significant_change[significant_change['tp3'] == significant_change['max']]
bin_4 = significant_change[significant_change['tp4'] == significant_change['max']]
bin_10 = significant_change[significant_change['tp5'] == significant_change['max']]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [76]:
print(bin_1[0:40].tp1)

A0A096YH13    2.252385
A0A096YH27    2.253413
A0A096YH64    2.477661
A0A096YHA3    4.291330
A0A096YHV8    3.520718
A0A096YHZ7    2.927715
A0A096YID1    2.099021
A0A096YID3    2.516877
A0A096YIF9    3.784585
A0A096YIK7    6.377166
A0A096YJD6    2.616800
A0A096YJH8    6.184265
A0A096YJJ4    4.689589
A0A096YJN4    2.219049
A0A096YKY5    2.971220
A0A096YLD0    2.270402
A0A096YLJ5    3.399054
A0A096YMI2    2.499268
A0A096YND3    1.846177
A0A096YNJ8    2.041791
A0A096YNL1    3.389439
A0A096YR23    2.142555
A0A096YR57    2.670941
A0A096YRK7    2.126209
A0A096YRV9    1.927593
A0A096YSG4    2.677079
A0A096YSH7    3.142589
A0A096YSI9    2.123948
A0A096YTT9    1.967050
A0A096YU19    2.123928
A0A096YUJ5    2.533260
A0A096YUT2    1.838481
A0A096YUT7    2.122474
A0A096YW89    2.279878
A0A096YX92    2.052742
Q2STS2        2.599447
Q2STT7        2.085929
Q2SUA0        2.142390
Q2SUF0        2.919044
Q2SUF6        3.730119
Name: tp1, dtype: float64


In [115]:
proteins_to_genes = pd.read_csv('https://raw.githubusercontent.com/BIO465/2019_Winter_Protein_Growth_Time_Analysis/master/data/bthai_prot_genes_map.txt', sep='\t', index_col = 0)

In [126]:
print(proteins_to_genes.join(bin_4, how='inner').Gene[0:60])


Q2STE4          atpE
Q2SUC0     BTH_I2975
Q2SUJ9     BTH_I2888
Q2SUS4     BTH_I2813
Q2SUS6     BTH_I2811
Q2SV20     BTH_I2715
Q2SV30     BTH_I2704
Q2SV42     BTH_I2692
Q2SV43     BTH_I2691
Q2SV61     BTH_I2673
Q2SV99     BTH_I2635
Q2SVF1     BTH_I2582
Q2SVP1     BTH_I2487
Q2SW04     BTH_I2374
Q2SW70     BTH_I2308
Q2SW91     BTH_I2287
Q2SWL1     BTH_I2167
Q2SXA8     BTH_I1911
Q2SXI8     BTH_I1829
Q2SXJ0     BTH_I1827
Q2SXP4     BTH_I1773
Q2SXX4     BTH_I1688
Q2SYQ5     BTH_I1398
Q2SYQ8     BTH_I1395
Q2SYR3     BTH_I1390
Q2SYT5     BTH_I1367
Q2SYT6     BTH_I1366
Q2SZ98     BTH_I1204
Q2SZK9     BTH_I1087
Q2SZL1     BTH_I1085
Q2T048     BTH_I0895
Q2T0K5     BTH_I0738
Q2T0Z1     BTH_I0601
Q2T178     BTH_I0514
Q2T180     BTH_I0512
Q2T2I2     BTH_I0053
Q2T2M0     BTH_I0082
Q2T380    BTH_II2181
Q2T391    BTH_II2170
Q2T3E2    BTH_II2119
Q2T3M6    BTH_II2030
Q2T3S0    BTH_II1985
Q2T3S2    BTH_II1983
Q2T471    BTH_II1834
Q2T473    BTH_II1832
Q2T480    BTH_II1825
Q2T4A2    BTH_II1803
Q2T4A3    BTH

In [15]:
f = open("bthai_bin_1.txt", "w")
for row in bin_1:
    f.write()
f.write(bin_1)
f = open("bthai_bin_2.txt", "w")
f.write(bin_2)
f = open("bthai_bin_3.txt", "w")
f.write(bin_3)
f = open("bthai_bin_4.txt", "w")
f.write(bin_4)
f = open("bthai_bin_10.txt", "w")
f.write(bin_10)

TypeError: write() argument must be str, not DataFrame

In [93]:
for element in bin_1.index:
    if element[0] != 'A':
        print (element)

Q2STS2
Q2STT7
Q2SUA0
Q2SUF0
Q2SUF6
Q2SVE4
Q2SVH5
Q2SVK3
Q2SWB6
Q2SWZ4
Q2SXD9
Q2SXE3
Q2SXE4
Q2SXR0
Q2SXT3
Q2SYJ4
Q2SZ51
Q2SZL6
Q2SZT4
Q2T085
Q2T0C7
Q2T0C8
Q2T0G0
Q2T0Y5
Q2T112
Q2T134
Q2T1Z8
Q2T2C7
Q2T318
Q2T3N6
Q2T7N8


## Create a simple chart to show the number of proteins in each category

In [9]:
bins = ['Peak TP1', 'Peak TP2', 'Peak TP3', 'Peak TP4', 'Peak TP5']
counts = [len(bin_1), len(bin_2), len(bin_3), len(bin_4), len(bin_10)]

p = figure(x_range = bins, plot_height = 500, title = 'Distribution of Protein Peaks',
           x_axis_label = 'Time Point Peak',
           y_axis_label = 'Number of Proteins')

p.vbar(x = bins, top = counts, width = 0.9, color = Viridis8[0:5])

p.xgrid.visible = False
p.ygrid.visible = False
p.title.text_font_size = '20pt'
p.title.align = 'center'
p.xaxis.axis_label_text_font_size = '15pt'
p.yaxis.axis_label_text_font_size = '15pt'
p.yaxis.minor_tick_line_color = None

show(p)

## Plot each category

In [10]:
p = figure(plot_width=400, plot_height=400)

for i in range(0, len(bin_rejects)):
    p.line([1, 2, 3, 4, 5], [bin_rejects['tp1'][i], bin_rejects['tp2'][i], bin_rejects['tp3'][i], bin_rejects['tp4'][i], bin_rejects['tp5'][i]])
    
p.xaxis.axis_label = 'Time Point'
p.yaxis.axis_label = 'Relative Protein Abundance'
p.xgrid.visible = False
p.ygrid.visible = False
p.xaxis.minor_tick_line_color = None
p.yaxis.minor_tick_line_color = None
    
show(p)

In [25]:
p = figure(plot_width=1000, plot_height=1000,
           x_axis_label = 'Growth Phase',
           y_axis_label = 'Relative Protein Abundance')

for i in range(0, len(bin_1)):
    p.line([1, 2, 3, 4, 5], [bin_1['tp1'][i], bin_1['tp2'][i], bin_1['tp3'][i], bin_1['tp4'][i], bin_1['tp5'][i]])
    
p.xgrid.visible = False
p.ygrid.visible = False
p.title.text_font_size = '32pt'
p.title.align = 'center'
p.xaxis.axis_label_text_font_size = '30pt'
p.yaxis.axis_label_text_font_size = '30pt'
p.xaxis.major_label_text_font_size = "25pt"
p.xaxis.major_label_overrides = {1: 'enter log', 2: 'mid log', 3: 'late log', 4: 'early stat.', 5: 'late stat.'}
p.yaxis.major_label_text_font_size = "15pt"
p.xaxis.minor_tick_line_color = None
p.yaxis.minor_tick_line_color = None
    
show(p)

In [27]:
p = figure(plot_width=1000, plot_height=1000,
           x_axis_label = 'Growth Phase',
           y_axis_label = 'Relative Protein Abundance')

for i in range(0, len(bin_2)):
    p.line([1, 2, 3, 4, 5], [bin_2['tp1'][i], bin_2['tp2'][i], bin_2['tp3'][i], bin_2['tp4'][i], bin_2['tp5'][i]])
    
p.xgrid.visible = False
p.ygrid.visible = False
p.title.text_font_size = '32pt'
p.title.align = 'center'
p.xaxis.axis_label_text_font_size = '30pt'
p.yaxis.axis_label_text_font_size = '30pt'
p.xaxis.major_label_text_font_size = "25pt"
p.xaxis.major_label_overrides = {1: 'enter log', 2: 'mid log', 3: 'late log', 4: 'early stat.', 5: 'late stat.'}
p.yaxis.major_label_text_font_size = "15pt"
p.xaxis.minor_tick_line_color = None
p.yaxis.minor_tick_line_color = None
    
show(p)

In [28]:
p = figure(plot_width=1000, plot_height=1000,
           x_axis_label = 'Growth Phase',
           y_axis_label = 'Relative Protein Abundance')

for i in range(0, len(bin_3)):
    p.line([1, 2, 3, 4, 5], [bin_3['tp1'][i], bin_3['tp2'][i], bin_3['tp3'][i], bin_3['tp4'][i], bin_3['tp5'][i]])
    
p.xgrid.visible = False
p.ygrid.visible = False
p.title.text_font_size = '32pt'
p.title.align = 'center'
p.xaxis.axis_label_text_font_size = '30pt'
p.yaxis.axis_label_text_font_size = '30pt'
p.xaxis.major_label_text_font_size = "25pt"
p.xaxis.major_label_overrides = {1: 'enter log', 2: 'mid log', 3: 'late log', 4: 'early stat.', 5: 'late stat.'}
p.yaxis.major_label_text_font_size = "15pt"
p.xaxis.minor_tick_line_color = None
p.yaxis.minor_tick_line_color = None
    
show(p)

In [29]:
p = figure(plot_width=1000, plot_height=1000,
           x_axis_label = 'Growth Phase',
           y_axis_label = 'Relative Protein Abundance')

for i in range(0, len(bin_4)):
    p.line([1, 2, 3, 4, 5], [bin_4['tp1'][i], bin_4['tp2'][i], bin_4['tp3'][i], bin_4['tp4'][i], bin_4['tp5'][i]])
    
p.xgrid.visible = False
p.ygrid.visible = False
p.title.text_font_size = '32pt'
p.title.align = 'center'
p.xaxis.axis_label_text_font_size = '30pt'
p.yaxis.axis_label_text_font_size = '30pt'
p.xaxis.major_label_text_font_size = "25pt"
p.xaxis.major_label_overrides = {1: 'enter log', 2: 'mid log', 3: 'late log', 4: 'early stat.', 5: 'late stat.'}
p.yaxis.major_label_text_font_size = "15pt"
p.xaxis.minor_tick_line_color = None
p.yaxis.minor_tick_line_color = None

show(p)

In [30]:
p = figure(plot_width=1000, plot_height=1000,
           x_axis_label = 'Growth Phase',
           y_axis_label = 'Relative Protein Abundance')

for i in range(0, len(bin_10)):
    p.line([1, 2, 3, 4, 5], [bin_10['tp1'][i], bin_10['tp2'][i], bin_10['tp3'][i], bin_10['tp4'][i], bin_10['tp5'][i]])
    
p.xgrid.visible = False
p.ygrid.visible = False
p.title.text_font_size = '32pt'
p.title.align = 'center'
p.xaxis.axis_label_text_font_size = '30pt'
p.yaxis.axis_label_text_font_size = '30pt'
p.xaxis.major_label_text_font_size = "25pt"
p.xaxis.major_label_overrides = {1: 'enter log', 2: 'mid log', 3: 'late log', 4: 'early stat.', 5: 'late stat.'}
p.yaxis.major_label_text_font_size = "15pt"
p.xaxis.minor_tick_line_color = None
p.yaxis.minor_tick_line_color = None
    
show(p)

## Combine all into a single plot using best-fit lines

In [16]:
bin_1.loc['mean'] = bin_1.mean()
bin_2.loc['mean'] = bin_2.mean()
bin_3.loc['mean'] = bin_3.mean()
bin_4.loc['mean'] = bin_4.mean()
bin_10.loc['mean'] = bin_10.mean()

columns = bin_1.columns
bestfit_lines = pd.DataFrame(columns = columns)
bestfit_lines.loc['bin1_mean'] = bin_1.loc['mean']
bestfit_lines.loc['bin2_mean'] = bin_2.loc['mean']
bestfit_lines.loc['bin3_mean'] = bin_3.loc['mean']
bestfit_lines.loc['bin4_mean'] = bin_4.loc['mean']
bestfit_lines.loc['bin10_mean'] = bin_10.loc['mean']

bestfit_lines

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/lib/python3.6/site-p

,tp1,tp2,tp3,tp4,tp5,max,min,diff_max_min
bin1_mean,2.756582,1.659259,1.085699,0.820617,0.754119,2.756582,0.615515,2.141067
bin2_mean,0.706127,2.575402,1.202299,1.068854,0.888895,2.575402,0.364515,2.210887
bin3_mean,0.601888,1.007972,3.064323,1.714618,1.433507,3.064323,0.480291,2.584032
bin4_mean,0.606185,0.952018,1.737892,2.891336,2.364472,2.891336,0.502127,2.389209
bin10_mean,0.581113,0.816192,1.560086,2.567547,3.527362,3.527362,0.505965,3.021397


In [17]:
p = figure(plot_width=1000, plot_height=1000,
           x_axis_label = 'Time Point',
           y_axis_label = 'Relative Protein Abundance')

for i in range(0, len(bestfit_lines)):
    p.line([1, 2, 3, 4, 5], [bestfit_lines['tp1'][i], bestfit_lines['tp2'][i], bestfit_lines['tp3'][i], bestfit_lines['tp4'][i], bestfit_lines['tp5'][i]], color=Viridis8[i], line_width=10)
    
p.xgrid.visible = False
p.ygrid.visible = False
p.title.text_font_size = '32pt'
p.title.align = 'center'
p.xaxis.axis_label_text_font_size = '30pt'
p.yaxis.axis_label_text_font_size = '30pt'
p.xaxis.major_label_text_font_size = "15pt"
p.yaxis.major_label_text_font_size = "15pt"
p.xaxis.minor_tick_line_color = None
p.yaxis.minor_tick_line_color = None

show(p)